In [1]:
%load_ext autoreload
%autoreload 2
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

<IPython.core.display.Javascript object>

In [3]:
try:
    %tensorflow_version 2.x
except Exception:
    pass

<IPython.core.display.Javascript object>

In [4]:
SETUP = False

<IPython.core.display.Javascript object>

In [5]:
if SETUP:
    !pip install -q -U toai
    !pip install -q -U nb_black
    !pip install -q -U tensorflow-datasets
    !pip install -q -U --no-deps tensorflow-addons
    !pip install -q -U tensorflow_hub
    print(__import__("toai").__version__)
    print(__import__("tensorflow").__version__)

<IPython.core.display.Javascript object>

In [6]:
print(__import__("toai").__version__)

0.3.7


<IPython.core.display.Javascript object>

In [7]:
from toai.imports import *
from toai.utils import save_file, load_file
from toai.data.utils import split_df
from toai.models import save_keras_model, load_keras_model
import tensorflow as tf
from tensorflow import keras
import tensorflow_addons as tfa

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


<IPython.core.display.Javascript object>

In [8]:
DATA_DIR = Path("data/wine-reviews")
TEMP_DIR = Path("temp/wine-reviews")

<IPython.core.display.Javascript object>

In [9]:
if SETUP:
    shutil.rmtree(str(DATA_DIR), ignore_errors=True)
    shutil.rmtree(str(TEMP_DIR), ignore_errors=True)
    DATA_DIR.mkdir(parents=True)
    TEMP_DIR.mkdir(parents=True)
    kaggle.api.authenticate()
    kaggle.api.dataset_download_files(
        dataset="zynicide/wine-reviews", path=DATA_DIR, unzip=True
    )

<IPython.core.display.Javascript object>

In [10]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 2048

<IPython.core.display.Javascript object>

In [11]:
all_data = pd.read_csv(
    DATA_DIR / "winemag-data-130k-v2.csv", index_col=0, low_memory=False
)

<IPython.core.display.Javascript object>

In [12]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 129971 entries, 0 to 129970
Data columns (total 13 columns):
country                  129908 non-null object
description              129971 non-null object
designation              92506 non-null object
points                   129971 non-null int64
price                    120975 non-null float64
province                 129908 non-null object
region_1                 108724 non-null object
region_2                 50511 non-null object
taster_name              103727 non-null object
taster_twitter_handle    98758 non-null object
title                    129971 non-null object
variety                  129970 non-null object
winery                   129971 non-null object
dtypes: float64(1), int64(1), object(11)
memory usage: 13.9+ MB


<IPython.core.display.Javascript object>

In [13]:
all_data.describe(include="all")

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
count,129908,129971,92506,129971.000000,120975.000000,129908,108724,50511,103727,98758,129971,129970,129971
unique,43,119955,37979,NaN,NaN,425,1229,17,19,15,118840,707,16757
top,US,"Ripe plum, game, truffle, leather and menthol ...",Reserve,NaN,NaN,California,Napa Valley,Central Coast,Roger Voss,@vossroger,Gloria Ferrer NV Sonoma Brut Sparkling (Sonoma...,Pinot Noir,Wines & Winemakers
freq,54504,3,2009,NaN,NaN,36247,4480,11065,25514,25514,11,13272,222
mean,NaN,NaN,NaN,88.447138,35.363389,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,3.039730,41.022218,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,80.000000,4.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,86.000000,17.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,88.000000,25.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,91.000000,42.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<IPython.core.display.Javascript object>

In [14]:
all_data.head(10).T

,0,1,2,3,4,5,6,7,8,9
country,Italy,Portugal,US,US,US,Spain,Italy,France,Germany,France
description,"Aromas include tropical fruit, broom, brimston...","This is ripe and fruity, a wine that is smooth...","Tart and snappy, the flavors of lime flesh and...","Pineapple rind, lemon pith and orange blossom ...","Much like the regular bottling from 2012, this...",Blackberry and raspberry aromas show a typical...,"Here's a bright, informal red that opens with ...",This dry and restrained wine offers spice in p...,Savory dried thyme notes accent sunnier flavor...,This has great depth of flavor with its fresh ...
designation,Vulkà Bianco,Avidagos,NaN,Reserve Late Harvest,Vintner's Reserve Wild Child Block,Ars In Vitro,Belsito,NaN,Shine,Les Natures
points,87,87,87,87,87,87,87,87,87,87
price,NaN,15,14,13,65,15,16,24,12,27
province,Sicily & Sardinia,Douro,Oregon,Michigan,Oregon,Northern Spain,Sicily & Sardinia,Alsace,Rheinhessen,Alsace
region_1,Etna,NaN,Willamette Valley,Lake Michigan Shore,Willamette Valley,Navarra,Vittoria,Alsace,NaN,Alsace
region_2,NaN,NaN,Willamette Valley,NaN,Willamette Valley,NaN,NaN,NaN,NaN,NaN
taster_name,Kerin O’Keefe,Roger Voss,Paul Gregutt,Alexander Peartree,Paul Gregutt,Michael Schachner,Kerin O’Keefe,Roger Voss,Anna Lee C. Iijima,Roger Voss
taster_twitter_handle,@kerinokeefe,@vossroger,@paulgwine,NaN,@paulgwine,@wineschach,@kerinokeefe,@vossroger,NaN,@vossroger


<IPython.core.display.Javascript object>

In [15]:
available_data = all_data[~all_data["price"].isna()]

<IPython.core.display.Javascript object>

In [16]:
available_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 120975 entries, 1 to 129970
Data columns (total 13 columns):
country                  120916 non-null object
description              120975 non-null object
designation              86196 non-null object
points                   120975 non-null int64
price                    120975 non-null float64
province                 120916 non-null object
region_1                 101400 non-null object
region_2                 50292 non-null object
taster_name              96479 non-null object
taster_twitter_handle    91559 non-null object
title                    120975 non-null object
variety                  120974 non-null object
winery                   120975 non-null object
dtypes: float64(1), int64(1), object(11)
memory usage: 12.9+ MB


<IPython.core.display.Javascript object>

In [17]:
data = available_data.fillna("0")

<IPython.core.display.Javascript object>

In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 120975 entries, 1 to 129970
Data columns (total 13 columns):
country                  120975 non-null object
description              120975 non-null object
designation              120975 non-null object
points                   120975 non-null int64
price                    120975 non-null float64
province                 120975 non-null object
region_1                 120975 non-null object
region_2                 120975 non-null object
taster_name              120975 non-null object
taster_twitter_handle    120975 non-null object
title                    120975 non-null object
variety                  120975 non-null object
winery                   120975 non-null object
dtypes: float64(1), int64(1), object(11)
memory usage: 12.9+ MB


<IPython.core.display.Javascript object>

In [19]:
data.columns.values

array(['country', 'description', 'designation', 'points', 'price',
       'province', 'region_1', 'region_2', 'taster_name',
       'taster_twitter_handle', 'title', 'variety', 'winery'],
      dtype=object)

<IPython.core.display.Javascript object>

In [20]:
cat_cols = [
    "country",
    "designation",
    "province",
    "region_1",
    "region_2",
    "taster_name",
    "taster_twitter_handle",
    "title",
    "variety",
    "winery",
]

<IPython.core.display.Javascript object>

In [21]:
cont_cols = ["points", "price"]

<IPython.core.display.Javascript object>

In [22]:
cat_feature_cols = cat_cols

<IPython.core.display.Javascript object>

In [23]:
cont_feature_cols = ["points"]

<IPython.core.display.Javascript object>

In [24]:
feature_cols = cat_feature_cols + cont_feature_cols

<IPython.core.display.Javascript object>

In [25]:
target_col = "price"

<IPython.core.display.Javascript object>

In [26]:
def make_category_map(labels):
    return {x: i for i, x in enumerate(sorted(set(labels)))}

<IPython.core.display.Javascript object>

In [27]:
def init_category_maps(filename, df, feature_cols):
    try:
        category_maps = load_file(filename)
    except:
        category_maps = {
            feature_col: make_category_map(df[feature_col].values)
            for feature_col in feature_cols
        }
        save_file(category_maps, filename)
    return category_maps

<IPython.core.display.Javascript object>

In [28]:
category_maps = init_category_maps(TEMP_DIR / "category_maps.pickle", data, cat_cols)

<IPython.core.display.Javascript object>

In [29]:
category_maps

{'country': {'0': 0,
  'Argentina': 1,
  'Armenia': 2,
  'Australia': 3,
  'Austria': 4,
  'Bosnia and Herzegovina': 5,
  'Brazil': 6,
  'Bulgaria': 7,
  'Canada': 8,
  'Chile': 9,
  'China': 10,
  'Croatia': 11,
  'Cyprus': 12,
  'Czech Republic': 13,
  'England': 14,
  'France': 15,
  'Georgia': 16,
  'Germany': 17,
  'Greece': 18,
  'Hungary': 19,
  'India': 20,
  'Israel': 21,
  'Italy': 22,
  'Lebanon': 23,
  'Luxembourg': 24,
  'Macedonia': 25,
  'Mexico': 26,
  'Moldova': 27,
  'Morocco': 28,
  'New Zealand': 29,
  'Peru': 30,
  'Portugal': 31,
  'Romania': 32,
  'Serbia': 33,
  'Slovakia': 34,
  'Slovenia': 35,
  'South Africa': 36,
  'Spain': 37,
  'Switzerland': 38,
  'Turkey': 39,
  'US': 40,
  'Ukraine': 41,
  'Uruguay': 42},
 'designation': {'#19 Phantom Limb Belmont Vineyard': 0,
  '#50 Mon Chou': 1,
  '#SocialSecret': 2,
  '%@#$!': 3,
  '&': 4,
  "'61 Rosé": 5,
  "'A Rina": 6,
  "'Blend 105' Red Wine": 7,
  "'Na Vota": 8,
  "'S'": 9,
  "'Unfiltered'": 10,
  "'Vie!'": 11,

<IPython.core.display.Javascript object>

In [30]:
def apply_category_maps(df, category_maps):
    df = df.copy()
    for feature_col, feature_map in category_maps.items():
        df[feature_col] = df[feature_col].map(feature_map)
    return df

<IPython.core.display.Javascript object>

In [31]:
target_scaler = sk.preprocessing.RobustScaler().fit(
    data[target_col].values.reshape(-1, 1)
)

<IPython.core.display.Javascript object>

In [32]:
save_file(target_scaler, TEMP_DIR / "target_scaler.pickle")

<IPython.core.display.Javascript object>

In [33]:
feature_scaler = sk.preprocessing.RobustScaler().fit(data[cont_feature_cols])

<IPython.core.display.Javascript object>

In [34]:
save_file(feature_scaler, TEMP_DIR / "feature_scaler.pickle")

<IPython.core.display.Javascript object>

In [35]:
train_data, val_data, test_data = split_df(data, 0.4)

<IPython.core.display.Javascript object>

In [36]:
train_data.shape, val_data.shape, test_data.shape

((72585, 13), (24195, 13), (24195, 13))

<IPython.core.display.Javascript object>

In [37]:
train_data = apply_category_maps(train_data, category_maps)

<IPython.core.display.Javascript object>

In [38]:
val_data = apply_category_maps(val_data, category_maps)

<IPython.core.display.Javascript object>

In [39]:
test_data = apply_category_maps(test_data, category_maps)

<IPython.core.display.Javascript object>

In [40]:
train_data[cont_feature_cols] = feature_scaler.transform(train_data[cont_feature_cols])

<IPython.core.display.Javascript object>

In [41]:
val_data[cont_feature_cols] = feature_scaler.transform(val_data[cont_feature_cols])

<IPython.core.display.Javascript object>

In [42]:
test_data[cont_feature_cols] = feature_scaler.transform(test_data[cont_feature_cols])

<IPython.core.display.Javascript object>

In [43]:
train_data[target_col] = target_scaler.transform(train_data[[target_col]])

<IPython.core.display.Javascript object>

In [44]:
val_data[target_col] = target_scaler.transform(val_data[[target_col]])

<IPython.core.display.Javascript object>

In [45]:
test_data[target_col] = target_scaler.transform(test_data[[target_col]])

<IPython.core.display.Javascript object>

In [46]:
train_dataset = (
    tf.data.Dataset.zip(
        (
            tf.data.Dataset.from_tensor_slices(
                tuple(
                    [train_data[cal_col].values for cal_col in cat_feature_cols]
                    + [train_data[cont_feature_cols].values]
                )
            ),
            tf.data.Dataset.from_tensor_slices(train_data[target_col].values),
        )
    )
    .repeat()
    .shuffle(len(train_data))
    .batch(BATCH_SIZE)
    .prefetch(-1)
)

<IPython.core.display.Javascript object>

In [47]:
val_dataset = (
    tf.data.Dataset.zip(
        (
            tf.data.Dataset.from_tensor_slices(
                tuple(
                    [val_data[cal_col].values for cal_col in cat_feature_cols]
                    + [val_data[cont_feature_cols].values]
                )
            ),
            tf.data.Dataset.from_tensor_slices(val_data[target_col].values),
        )
    )
    .batch(BATCH_SIZE)
    .prefetch(-1)
)

<IPython.core.display.Javascript object>

In [48]:
train_data_steps = len(train_data) // BATCH_SIZE

<IPython.core.display.Javascript object>

In [49]:
def make_embedding_model(
    cat_feature_dims,
    n_cont_features,
    layer_size=256,
    dropout_rate=0.0,
    regularization=None,
    n_layers=1,
    batch_norm=True,
):

    inputs = []
    embeddings = []
    for cat_feature_dim in cat_feature_dims:
        x = keras.layers.Input(shape=(1,))
        inputs.append(x)
        x = keras.layers.Embedding(
            cat_feature_dim,
            min(50, np.sqrt(cat_feature_dim).astype(int) + 1),
            input_length=1,
            embeddings_regularizer=keras.regularizers.l1_l2(1e-2, 1e-2),
        )(x)
        x = keras.layers.Flatten()(x)
        embeddings.append(x)

    cont_inputs = keras.layers.Input(shape=(n_cont_features,))
    inputs.append(cont_inputs)
    embeddings.append(cont_inputs)

    x = keras.layers.Concatenate()(embeddings)

    for _ in range(n_layers):
        if batch_norm:
            x = keras.layers.BatchNormalization()(x)
        x = keras.layers.Dropout(dropout_rate)(x)
        x = keras.layers.Dense(
            layer_size,
            kernel_initializer=keras.initializers.he_uniform(),
            kernel_regularizer=regularization
            and keras.regularizers.l1_l2(*regularization),
        )(x)
        x = keras.layers.LeakyReLU()(x)

    x = keras.layers.Dense(1)(x)

    model = keras.Model(inputs=inputs, outputs=x)
    return model

<IPython.core.display.Javascript object>

In [50]:
embedding_model = make_embedding_model(
    [len(x) + 1 for x in category_maps.values()],
    1,
    layer_size=256,
    dropout_rate=0.5,
    regularization=(1e-8, 3e-3),
    n_layers=3,
)

<IPython.core.display.Javascript object>

In [51]:
embedding_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 1)]          0                                            
______________________________________________________________________________________________

<IPython.core.display.Javascript object>

In [52]:
embedding_model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss=keras.losses.mse,
    metrics=[keras.metrics.mae],
)

<IPython.core.display.Javascript object>

In [53]:
PATIENCE = 20

embedding_model_history = embedding_model.fit(
    train_dataset,
    steps_per_epoch=train_data_steps,
    validation_data=val_dataset,
    epochs=1000,
    callbacks=[
        tf.keras.callbacks.ReduceLROnPlateau(factor=0.3, patience=PATIENCE),
        tf.keras.callbacks.EarlyStopping(
            patience=PATIENCE * 2, restore_best_weights=True
        ),
    ],
    verbose=2,
)

Train for 35 steps, validate for 12 steps
Epoch 1/1000


/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


35/35 - 7s - loss: 1043.2083 - mean_absolute_error: 1.0189 - val_loss: 262.4180 - val_mean_absolute_error: 0.6840
Epoch 2/1000
35/35 - 3s - loss: 94.9989 - mean_absolute_error: 0.7970 - val_loss: 31.9441 - val_mean_absolute_error: 0.6990
Epoch 3/1000
35/35 - 3s - loss: 21.7404 - mean_absolute_error: 0.7557 - val_loss: 17.7357 - val_mean_absolute_error: 0.7249
Epoch 4/1000
35/35 - 3s - loss: 17.3290 - mean_absolute_error: 0.7327 - val_loss: 16.6912 - val_mean_absolute_error: 0.7465
Epoch 5/1000
35/35 - 3s - loss: 16.0667 - mean_absolute_error: 0.7093 - val_loss: 16.3844 - val_mean_absolute_error: 0.7365
Epoch 6/1000
35/35 - 2s - loss: 15.9812 - mean_absolute_error: 0.7110 - val_loss: 16.6480 - val_mean_absolute_error: 0.7599
Epoch 7/1000
35/35 - 2s - loss: 15.8309 - mean_absolute_error: 0.6768 - val_loss: 16.4915 - val_mean_absolute_error: 0.7138
Epoch 8/1000
35/35 - 2s - loss: 15.5242 - mean_absolute_error: 0.6681 - val_loss: 16.2562 - val_mean_absolute_error: 0.7480
Epoch 9/1000
35/35

Epoch 67/1000
35/35 - 2s - loss: 13.4589 - mean_absolute_error: 0.4982 - val_loss: 13.9981 - val_mean_absolute_error: 0.4720
Epoch 68/1000
35/35 - 2s - loss: 13.3174 - mean_absolute_error: 0.4869 - val_loss: 13.9165 - val_mean_absolute_error: 0.4863
Epoch 69/1000
35/35 - 2s - loss: 7.6399 - mean_absolute_error: 0.4798 - val_loss: 6.6831 - val_mean_absolute_error: 0.4834
Epoch 70/1000
35/35 - 2s - loss: 5.6128 - mean_absolute_error: 0.4513 - val_loss: 6.4587 - val_mean_absolute_error: 0.4881
Epoch 71/1000
35/35 - 2s - loss: 5.5237 - mean_absolute_error: 0.4415 - val_loss: 6.4518 - val_mean_absolute_error: 0.4890
Epoch 72/1000
35/35 - 2s - loss: 5.5211 - mean_absolute_error: 0.4452 - val_loss: 6.2611 - val_mean_absolute_error: 0.4834
Epoch 73/1000
35/35 - 2s - loss: 5.5934 - mean_absolute_error: 0.4439 - val_loss: 6.4922 - val_mean_absolute_error: 0.4818
Epoch 74/1000
35/35 - 2s - loss: 5.4804 - mean_absolute_error: 0.4378 - val_loss: 6.3054 - val_mean_absolute_error: 0.4725
Epoch 75/100

Epoch 134/1000
35/35 - 2s - loss: 1.3659 - mean_absolute_error: 0.3436 - val_loss: 2.2287 - val_mean_absolute_error: 0.4246
Epoch 135/1000
35/35 - 2s - loss: 1.3898 - mean_absolute_error: 0.3441 - val_loss: 2.2263 - val_mean_absolute_error: 0.4246
Epoch 136/1000
35/35 - 2s - loss: 1.3723 - mean_absolute_error: 0.3425 - val_loss: 2.2102 - val_mean_absolute_error: 0.4192
Epoch 137/1000
35/35 - 2s - loss: 1.4007 - mean_absolute_error: 0.3415 - val_loss: 2.2325 - val_mean_absolute_error: 0.4213
Epoch 138/1000
35/35 - 2s - loss: 1.4016 - mean_absolute_error: 0.3401 - val_loss: 2.2534 - val_mean_absolute_error: 0.4203
Epoch 139/1000
35/35 - 2s - loss: 1.3901 - mean_absolute_error: 0.3411 - val_loss: 2.2379 - val_mean_absolute_error: 0.4120
Epoch 140/1000
35/35 - 2s - loss: 1.3659 - mean_absolute_error: 0.3376 - val_loss: 2.2476 - val_mean_absolute_error: 0.4134
Epoch 141/1000
35/35 - 2s - loss: 1.3980 - mean_absolute_error: 0.3399 - val_loss: 2.2666 - val_mean_absolute_error: 0.4103
Epoch 14

...

Epoch 400/1000
35/35 - 2s - loss: 0.6033 - mean_absolute_error: 0.2857 - val_loss: 1.4711 - val_mean_absolute_error: 0.3820
Epoch 401/1000
35/35 - 2s - loss: 0.5994 - mean_absolute_error: 0.2781 - val_loss: 1.4660 - val_mean_absolute_error: 0.3814
Epoch 402/1000
35/35 - 2s - loss: 0.6013 - mean_absolute_error: 0.2792 - val_loss: 1.4662 - val_mean_absolute_error: 0.3815
Epoch 403/1000
35/35 - 2s - loss: 0.6061 - mean_absolute_error: 0.2799 - val_loss: 1.4671 - val_mean_absolute_error: 0.3822
Epoch 404/1000
35/35 - 2s - loss: 0.5912 - mean_absolute_error: 0.2815 - val_loss: 1.4669 - val_mean_absolute_error: 0.3816
Epoch 405/1000
35/35 - 2s - loss: 0.5931 - mean_absolute_error: 0.2783 - val_loss: 1.4642 - val_mean_absolute_error: 0.3813
Epoch 406/1000
35/35 - 2s - loss: 0.5901 - mean_absolute_error: 0.2835 - val_loss: 1.4651 - val_mean_absolute_error: 0.3808
Epoch 407/1000
35/35 - 2s - loss: 0.6151 - mean_absolute_error: 0.2833 - val_loss: 1.4671 - val_mean_absolute_error: 0.3814
Epoch 40

35/35 - 2s - loss: 0.5817 - mean_absolute_error: 0.2766 - val_loss: 1.4490 - val_mean_absolute_error: 0.3796
Epoch 467/1000
35/35 - 2s - loss: 0.5833 - mean_absolute_error: 0.2793 - val_loss: 1.4549 - val_mean_absolute_error: 0.3800
Epoch 468/1000
35/35 - 2s - loss: 0.5695 - mean_absolute_error: 0.2748 - val_loss: 1.4521 - val_mean_absolute_error: 0.3798
Epoch 469/1000
35/35 - 2s - loss: 0.5798 - mean_absolute_error: 0.2770 - val_loss: 1.4522 - val_mean_absolute_error: 0.3797
Epoch 470/1000
35/35 - 2s - loss: 0.6164 - mean_absolute_error: 0.2817 - val_loss: 1.4540 - val_mean_absolute_error: 0.3799
Epoch 471/1000
35/35 - 2s - loss: 0.5706 - mean_absolute_error: 0.2783 - val_loss: 1.4553 - val_mean_absolute_error: 0.3799
Epoch 472/1000
35/35 - 2s - loss: 0.5764 - mean_absolute_error: 0.2782 - val_loss: 1.4555 - val_mean_absolute_error: 0.3805
Epoch 473/1000
35/35 - 2s - loss: 0.5544 - mean_absolute_error: 0.2761 - val_loss: 1.4561 - val_mean_absolute_error: 0.3802
Epoch 474/1000
35/35 - 

Epoch 533/1000
35/35 - 2s - loss: 0.5733 - mean_absolute_error: 0.2789 - val_loss: 1.4338 - val_mean_absolute_error: 0.3805
Epoch 534/1000
35/35 - 2s - loss: 0.5664 - mean_absolute_error: 0.2813 - val_loss: 1.4346 - val_mean_absolute_error: 0.3805
Epoch 535/1000
35/35 - 2s - loss: 0.6004 - mean_absolute_error: 0.2836 - val_loss: 1.4379 - val_mean_absolute_error: 0.3807
Epoch 536/1000
35/35 - 2s - loss: 0.5696 - mean_absolute_error: 0.2821 - val_loss: 1.4363 - val_mean_absolute_error: 0.3804
Epoch 537/1000
35/35 - 2s - loss: 0.6043 - mean_absolute_error: 0.2821 - val_loss: 1.4323 - val_mean_absolute_error: 0.3802
Epoch 538/1000
35/35 - 2s - loss: 0.6028 - mean_absolute_error: 0.2832 - val_loss: 1.4320 - val_mean_absolute_error: 0.3804
Epoch 539/1000
35/35 - 2s - loss: 0.5602 - mean_absolute_error: 0.2801 - val_loss: 1.4322 - val_mean_absolute_error: 0.3806
Epoch 540/1000
35/35 - 2s - loss: 0.5576 - mean_absolute_error: 0.2834 - val_loss: 1.4335 - val_mean_absolute_error: 0.3811
Epoch 54

35/35 - 2s - loss: 0.5583 - mean_absolute_error: 0.2827 - val_loss: 1.4258 - val_mean_absolute_error: 0.3804
Epoch 600/1000
35/35 - 2s - loss: 0.5747 - mean_absolute_error: 0.2831 - val_loss: 1.4280 - val_mean_absolute_error: 0.3808
Epoch 601/1000
35/35 - 2s - loss: 0.6223 - mean_absolute_error: 0.2860 - val_loss: 1.4293 - val_mean_absolute_error: 0.3811
Epoch 602/1000
35/35 - 2s - loss: 0.5370 - mean_absolute_error: 0.2806 - val_loss: 1.4262 - val_mean_absolute_error: 0.3805
Epoch 603/1000
35/35 - 2s - loss: 0.5501 - mean_absolute_error: 0.2805 - val_loss: 1.4250 - val_mean_absolute_error: 0.3803
Epoch 604/1000
35/35 - 2s - loss: 0.5455 - mean_absolute_error: 0.2829 - val_loss: 1.4283 - val_mean_absolute_error: 0.3811
Epoch 605/1000
35/35 - 2s - loss: 0.5653 - mean_absolute_error: 0.2812 - val_loss: 1.4261 - val_mean_absolute_error: 0.3807
Epoch 606/1000
35/35 - 2s - loss: 0.5391 - mean_absolute_error: 0.2787 - val_loss: 1.4238 - val_mean_absolute_error: 0.3801
Epoch 607/1000
35/35 - 

<IPython.core.display.Javascript object>

In [54]:
random_forest_model = sk.model_selection.RandomizedSearchCV(
    estimator=Pipeline([("rf", RandomForestRegressor(n_estimators=100))]),
    param_distributions={
        "rf__max_depth": np.arange(10, 1000),
        "rf__min_samples_leaf": np.arange(3, 20),
        "rf__max_features": ["sqrt", "log2", 0.2],
        "rf__min_samples_split": np.arange(2, 20),
        "rf__min_samples_leaf": np.arange(1, 10),
    },
    n_iter=500,
    cv=3,
    n_jobs=-1,
    verbose=1,
)

<IPython.core.display.Javascript object>

In [55]:
random_forest_model.fit(train_data[feature_cols], train_data[target_col])

Fitting 3 folds for each of 500 candidates, totalling 1500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done 136 tasks      | elapsed:   48.6s
[Parallel(n_jobs=-1)]: Done 386 tasks      | elapsed:  1.9min
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done 736 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done 1186 tasks      | elapsed:  5.5min
[Parallel(n_jobs=-1)]: Done 1500 out of 1500 | elapsed:  6.9min finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=Pipeline(memory=None,
                                      steps=[('rf',
                                              RandomForestRegressor(bootstrap=True,
                                                                    criterion='mse',
                                                                    max_depth=None,
                                                                    max_features='auto',
                                                                    max_leaf_nodes=None,
                                                                    min_impurity_decrease=0.0,
                                                                    min_impurity_split=None,
                                                                    min_samples_leaf=1,
                                                                    min_samples_split=2,
                                           

<IPython.core.display.Javascript object>

In [56]:
val_predictions = {
    "nn": target_scaler.inverse_transform(embedding_model.predict(val_dataset)),
    "rf": target_scaler.inverse_transform(
        random_forest_model.predict(val_data[feature_cols]).reshape(-1, 1)
    ),
}

<IPython.core.display.Javascript object>

In [57]:
val_targets = target_scaler.inverse_transform(val_data[[target_col]])

<IPython.core.display.Javascript object>

In [58]:
from toai.metrics import rmse

<IPython.core.display.Javascript object>

In [65]:
metrics = [rmse, sk.metrics.mean_absolute_error]

<IPython.core.display.Javascript object>

In [66]:
for metric in metrics:
    for model_type, y_pred in val_predictions.items():
        print(f"{metric.__name__} [{model_type}]: {metric(val_targets, y_pred):.4f}")

rmse [nn]: 26.2130
rmse [rf]: 27.3319
mean_absolute_error [nn]: 9.4936
mean_absolute_error [rf]: 10.2045


<IPython.core.display.Javascript object>